- Promediar las vtas de agosto 2019 (201908) como las de julio (201907) y septiembre (201909) para todas las observaciones
- Buscar los 'product_id' que tengan poca hitoria (agrupandolos por product_id y periodo y validar que tengan menos registros que training_trashold), eliminarlos del conjunto, y agregarlos el un dataframe "Predicciones", poniendo product_id junto con una columna "prediccion", que sea la media de las ventas de los periodos
- Aplicar LabelEncoder a las columnas categoricas
- Agrupar los restantes las ventas por periodo, cat1, cat2, cat3, marca y descripcion
- Calcular para estos el ratio de ventas por product_id (para cada grupo de cat1, cat2, cat3, marca y descripcion), guardando esto en un diccionario: cat1, cat2, cat3, marca, descripcion, product_id y ratio

----
- Agrupar las ventas por periodo, cat1, cat2, cat3, marca, descripcion y customer_id. Sumarizando los valores de las columnas cust_request_qty, cust_request_tn y tn.
- Aplicar escalers por columna a cada grupo (guardando estos scalers en un diccionario)
- Armar un modelo LSTM para predecir las ventas de cada uno de estos grupos (usando todas las observaciones menos las ultimas 2 para predecir la ultima )
----

- Luego, para cada grupo, hacer las predicciones con su modelo correspondiente (usando todas las observaciones menos las primeras 2). Guardando estas predicciones en un dataframe con la estructura cat1, cat2, cat3, marca, descripcion
- sumarizar las predicciones por cat1, cat2, cat3, marca, descripcion
- para cada cat1, cat2, cat3, marca, descripcion, buscar los product_id en el diccionario de ratios, aplicarlo sobre las predicciones sumarizadas, y armar un dataframe product_id y prediccion
- unificar este dataframe con el "Predicciones"
- guardar este df en un csv

#### Imports

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

In [ ]:
df = pd.read_csv('../../Datasets/final_dataset_descr.csv', sep='\t')

In [ ]:
df.head()

#### Paso 2: Promediar las ventas de agosto 2019 (201908) con julio (201907) y septiembre (201909)

In [ ]:
df['periodo'] = df['periodo'].astype(str).str.strip()

# Filtrar los datos por los periodos 201907, 201908 y 201909
df_filtered = df[df['periodo'].isin(['201907', '201908', '201909'])]

# # Pivotear los datos para tener columnas separadas para cada periodo
pivoted_sales = df_filtered.pivot_table(index=['product_id', 'customer_id'], columns='periodo', values='tn').reset_index()

# # Asegurar que las columnas 201907 y 201909 existen en el DataFrame
pivoted_sales = pivoted_sales.reindex(columns=['product_id', 'customer_id', '201907', '201908', '201909'])

# # Calcular el promedio de julio y septiembre
pivoted_sales['201908'] = pivoted_sales[['201907', '201909']].mean(axis=1)

# # Convertir de nuevo al formato largo
updated_sales = pivoted_sales.melt(id_vars=['product_id', 'customer_id'], value_vars=['201907', '201908', '201909'], 
                                   var_name='periodo', value_name='tn')

# # Unir con el dataframe original
df.set_index(['product_id', 'customer_id', 'periodo'], inplace=True)
df.update(updated_sales.set_index(['product_id', 'customer_id', 'periodo']))
df.reset_index(inplace=True)

display(df)

#### Paso 3: Filtrar y eliminar productos con poca historia


In [ ]:
training_threshold = 12

# Contar el número de registros por product_id, customer_id y periodo
product_history = df.groupby(['product_id', 'customer_id']).size().reset_index(name='counts')

# Filtrar productos por customer_id con menos registros que el threshold
products_to_keep = product_history[product_history['counts'] >= training_threshold][['product_id', 'customer_id']].drop_duplicates()
df_filtered = df.merge(products_to_keep, on=['product_id', 'customer_id'], how='inner')

# Crear el DataFrame "Predicciones" para productos con poca historia por customer_id
products_to_predict = product_history[product_history['counts'] < training_threshold][['product_id', 'customer_id']].drop_duplicates()
predicciones = df.merge(products_to_predict, on=['product_id', 'customer_id'], how='inner').groupby(['product_id', 'customer_id'])['tn'].mean().reset_index()
predicciones.rename(columns={'tn': 'prediccion'}, inplace=True)


#### Paso 4: Aplicar LabelEncoder a las columnas categóricas


In [ ]:
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'quarter']

# Aplicar LabelEncoder
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_filtered[col] = le.fit_transform(df_filtered[col])
    label_encoders[col] = le


#### Paso 5: Agrupar y calcular el ratio de ventas por product_id


In [ ]:
# Agrupar por las columnas relevantes
grouped_sales = df_filtered.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'product_id'])['tn'].sum().reset_index()

# Calcular el total de ventas por grupo
group_totals = grouped_sales.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand'])['tn'].sum().reset_index()

# Unir para calcular el ratio
ratios = pd.merge(grouped_sales, group_totals, on=['periodo', 'cat1', 'cat2', 'cat3', 'brand'], suffixes=('', '_total'))

# Calcular el ratio
ratios['ratio'] = ratios['tn'] / ratios['tn_total']

# Crear un diccionario de ratios
ratio_dict = ratios.set_index(['cat1', 'cat2', 'cat3', 'brand', 'product_id'])['ratio'].to_dict()


#### Paso 6: Agrupar ventas por periodo, cat1, cat2, cat3, brand, descripcion y customer_id


In [ ]:
# Agrupar y sumarizar
grouped_df = df_filtered.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'quarter', 'month']).agg({
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()


#### Paso 7: Aplicar escalers por columna a cada grupo


In [ ]:
# Crear un diccionario para almacenar los scalers
scalers = {}
scaled_df = grouped_df.copy()

# Aplicar StandardScaler a cada columna de interés
for col in ['cust_request_qty', 'cust_request_tn', 'tn']:
    scaler = StandardScaler()
    scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    scalers[col] = scaler

# Guardar los scalers para su uso posterior
joblib.dump(scalers, 'scalers.pkl')


#### Paso 8: Armar un modelo LSTM


In [ ]:
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, GRU
from keras.regularizers import l2
from sklearn.model_selection import KFold
import numpy as np
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='tanh', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.1))
    # model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    # model.add(Dropout(0.1))
    # model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    # model.add(Dropout(0.1))
    # model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    # model.add(Dropout(0.1))
    # model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    # model.add(Dropout(0.1))
    # model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    # model.add(Dropout(0.1))
    model.add(LSTM(10, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

#### Paso 9: Entrenar y predecir con el modelo LSTM para cada grupo


In [ ]:
grouped = grouped_df.groupby(['cat1', 'cat2', 'cat3', 'customer_id', 'brand'])

# Obtener la lista de grupos
groups_list = list(grouped.groups.keys())

group_key = groups_list[29]

# Obtener el contenido del grupo específico
group_content = grouped.get_group(group_key)

group_content

In [ ]:
display(len(grouped_df.groupby(['cat1', 'cat2', 'cat3', 'customer_id', 'brand'])))

In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
from joblib import Parallel, delayed

grouped_df['periodo'] = pd.to_datetime(grouped_df['periodo'], format='%Y%m')

# Crear un diccionario para almacenar los modelos por grupo y una lista para predicciones
models = {}
predictions = []
cont = 0

# Agrupar y ordenar el DataFrame
groups = grouped_df.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id'])

# Función para procesar cada grupo
def process_group(group_key, group_data, cont):
    cat1, cat2, cat3, brand, customer_id = group_key
    
    # Ordenar por periodo
    group_data = group_data.sort_values(by='periodo')
    
    # Transformar los datos para LSTM
    n_steps = 2  # Por ejemplo
    X = group_data[['cust_request_qty', 'cust_request_tn', 'quarter', 'month', 'tn']].values
    y = group_data['tn'].values
    
    
    X_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))  # (número de muestras, timesteps, características)
    
    # Construir y entrenar el modelo
    model = build_lstm_model((X_reshaped.shape[1], X_reshaped.shape[2]))
    model.fit(X_reshaped[:-n_steps], y[n_steps:], epochs=40, verbose=0)
    
    # Hacer predicciones
    X_pred = group_data[['cust_request_qty', 'cust_request_tn', 'quarter', 'month', 'tn']].values[-n_steps:]  # Tomar los últimos n_steps
    X_pred = np.reshape(X_pred, (1, X_pred.shape[0], X_pred.shape[1]))  # Asegurar la forma correcta para la predicción
    pred = model.predict(X_pred, verbose=0)
    
    print(f"Predicción {cont}: {pred[0][0]}")
    
    return (cat1, cat2, cat3, brand, customer_id, pred[0][0])

# Paralelizar el procesamiento de los grupos
results = Parallel(n_jobs=-1, backend='threading')(
    delayed(process_group)(group_key, group_data, i)
    for i, (group_key, group_data) in enumerate(groups)
)

# Almacenar los resultados
for result in results:
    cat1, cat2, cat3, brand, customer_id, prediction = result
    predictions.append([cat1, cat2, cat3, brand, customer_id, prediction])

# Mostrar las predicciones
predictions_df = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'prediction'])
print(predictions_df)

#### Paso 10: Sumarizar las predicciones y aplicar ratios


In [ ]:
# Convertir las predicciones a un DataFrame
pred_df = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'prediccion', 'customer_id'])
pred_df = pd.concat([pred_df, predicciones])

# Sumarizar las predicciones por grupo
summarized_preds = pred_df.groupby(['cat1', 'cat2', 'cat3', 'brand'])['prediccion'].sum().reset_index()

# Aplicar los ratios para obtener las predicciones finales por product_id
final_predictions = []
for _, row in summarized_preds.iterrows():
    key = (row['cat1'], row['cat2'], row['cat3'], row['brand'])
    for (cat1, cat2, cat3, brand, product_id), ratio in ratio_dict.items():
        if (cat1, cat2, cat3, brand) == key:
            final_predictions.append([product_id, row['prediccion'] * ratio])

# Convertir las predicciones finales a un DataFrame
final_predictions_df = pd.DataFrame(final_predictions, columns=['product_id', 'prediccion'])

# Guardar el resultado en un archivo CSV
final_predictions_df.to_csv('predicciones_finales.csv', index=False)
